In [1]:
import re
import requests
import os
import pandas as pd
import numpy as np 

from lxml.etree import Element, SubElement, tostring, CDATA

In [2]:
# открываем из экселек 

price = pd.read_csv('price.csv', encoding='windows-1251', sep=';')
price = price.rename(columns={'Номенклатура': 'name'})
price.name = price.name.str.strip()
price['Цена'] = price['Цена'].str.replace(u'\xa0', u' ')
price['Цена'] = price['Цена'].str.replace(u' ', u'')
price['Цена'] = price['Цена'].str.replace(u',', u'.')
price['Остаток'] = price['Остаток'].str.replace(u'\xa0', u' ')
price['Остаток'] = price['Остаток'].str.replace(u' ', u'')
price['Остаток'] = price['Остаток'].str.replace(u',', u'.')
price['Цена'].fillna('0', inplace=True)
price['Цена'] = price['Цена'].astype('float')
price['Цена'] = np.floor(price['Цена'])
price['Цена'] = price['Цена'].astype('int')
# price = price[price['Цена'] >= 20000]


catalog = pd.read_excel('catalog.xlsx')
catalog.name = catalog.name.str.strip()
catalog.description.fillna('', inplace=True)
catalog.description = catalog.description.str.strip()


pics = pd.read_excel('pics.xlsx')
pics.name = pics.name.str.strip()

df=pd.merge(catalog, price, on='name', how='inner')

In [3]:
ads = Element('Ads')
ads.set('formatVersion', '3')
ads.set('target', 'Avito.ru')

In [4]:
regexp = ',*\s*(\d+)\s*шт/уп'
desc_footer = '<br/><br/><strong> При заказе от 2000 рублей бесплатная доставка по Казани в этот же день. Оплата при получении. <br/> Звоните и пишите! </strong>'

In [5]:
for i in range(df.shape[0]):
    ad = SubElement(ads, 'Ad')
    
    elem = SubElement(ad, 'Id')
    elem.text = df['Артикул'][i]
    
    elem = SubElement(ad, 'Address')
    elem.text = 'Россия, Казань'
    
    elem = SubElement(ad, 'Category')
    elem.text = 'Растения'
    
    
#     обработка шт/уп:
    amnt = 1
    name = df['name'][i]
    many = re.search(regexp, name)
    if many:
        amnt = int(many.group(1))
        name = re.sub(regexp, '', name)
        
    elem = SubElement(ad, 'Title')
    elem.text = name 
    
    elem = SubElement(ad, 'Description')
    elem.text = CDATA(df['description'][i] + desc_footer) 
    
    elem = SubElement(ad, 'Price')
    elem.text = str(int(df['Цена'][i]/amnt))
    
    elem = SubElement(ad, 'AdType')
    elem.text = 'Товар приобретен на продажу'
    
    images = SubElement(ad, 'Images')
    
    url_count = 0
    for url in pics[pics.name==df['name'][i]]['img_url']:
        image = SubElement(images, 'Image')
        image.set('url', url)
        url_count +=1
        if url_count == 5:
            break
        

In [6]:
out = tostring(ads, pretty_print=True, encoding=str)

with open('avito.xml', 'w', encoding='utf-8') as f:
    f.write(out)